In [1]:
from bglog import BGLog, get_embedding_layer
import numpy as np
import tensorflow as tf
tf.random.set_seed(123)

In [2]:
bglog = BGLog(save_padded_num_sequences=False, load_from_pkl=True)

In [3]:
train_test = bglog.get_tensor_train_test(ablation=1000)
train_data, test_data = train_test

padded_num_seq_df loaded from data\bgl_padded_num_seq_df.pkl
trained tokenizer, tk, loaded from data\bgltk.pkl
train_0:, 800
test_0:, 200
train_1:, 800
test_1:, 200
train_2:, 800
test_2:, 200
train_3:, 800
test_3:, 102
4 class does not have 800 records, it has only 628 records
test_4:, 0
5 class does not have 800 records, it has only 165 records
5 class does not have 200 records, it has only 165 records
6 class does not have 800 records, it has only 75 records
6 class does not have 200 records, it has only 75 records
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]]
<BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>


In [4]:
def model(conv1d_set1 = 3, conv1d_set2 = 3, dense_neurons=2048, filters=64,
            kernel_size=3,maxpool_1=True,epochs=25, dense_activation='relu'):
    embedding_weights, vocab_size, char_onehot = get_embedding_layer(bglog)
    B = train_data.element_spec[0].shape[0]
#     inputs = tf.keras.layers.Input(batch_shape=(B, train_data.element_spec[0].shape[1], train_data.element_spec[0].shape[2]), dtype='float64' )
    inputs = tf.keras.layers.Input(shape=(train_data.element_spec[0].shape[1], train_data.element_spec[0].shape[2]), dtype='float64' )
    x = tf.keras.layers.Embedding(input_dim=vocab_size+1,
                                    output_dim=vocab_size,
                                    input_length=train_data.element_spec[0].shape[2],
                                    weights = [embedding_weights],
                                    )(inputs)
    for _ in range(conv1d_set1):
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    if maxpool_1:
        x = tf.keras.layers.MaxPooling2D(pool_size=(1, train_data.element_spec[0].shape[2]))(x)
        x = tf.reshape(x, (B, train_data.element_spec[0].shape[1], filters))        
        for _ in range(conv1d_set2):
            x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
        x = tf.keras.layers.MaxPooling1D(pool_size=(train_data.element_spec[0].shape[1]) )(x)
        x = tf.reshape(x, (B, filters))
    if not maxpool_1:
        x = tf.keras.layers.Flatten()(x)
    if dense_activation is None:
        x = tf.keras.layers.Dense(dense_neurons)(x)
    else:
        x = tf.keras.layers.Dense(dense_neurons, activation=dense_activation)(x)
    outputs = tf.keras.layers.Dense(train_data.element_spec[1].shape[1], activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    print(model.summary())
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    hist = model.fit(train_data, validation_data=test_data, epochs=epochs) 
    return model, hist

In [5]:
# we  feed  xi  to  a dense layer h to get the log-sequence representation zi∈RD:
#     zi= h(xi) =σ(Whxi+bh) ............................(2)
# in our case zi can be obtained from the dense layer before the softmax
# Lets see how to ger it from the train mode

In [6]:
# we pre-train the model with labeled known intent samples. 
# In order to better reflect the effectiveness of the learned decision boundary, 
# we learn the feature representation zi with the simple softmax loss Ls to perform classification:

trained_model, hist = model(epochs=6,)

vocab_size: 50
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 64)]          0         
_________________________________________________________________
embedding (Embedding)        (None, 32, 64, 50)        2550      
_________________________________________________________________
conv1d (Conv1D)              (None, 32, 64, 64)        9664      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 32, 64, 64)        12352     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 32, 64, 64)        12352     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 1, 64)         0         
_________________________________________________________________
tf_op_layer_Reshape (TensorF [(32, 32, 

In [43]:
trained_model, hist = model(epochs=6, dense_neurons=64)

vocab_size: 50
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 64)]          0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 32, 64, 50)        2550      
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 32, 64, 64)        9664      
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 32, 64, 64)        12352     
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 32, 64, 64)        12352     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 1, 64)         0         
_________________________________________________________________
tf_op_layer_Reshape_2 (Tenso [(32, 32, 

In [7]:
# Learn the decision boundary of each class constraining the known labels within a ball area
# how to get zi and how to know that zi belongs to which yi ?
# from there we will have to calculate the Ck , centroid for the class k

In [8]:
trained_model.layers

In [11]:
dense_6 = trained_model.get_layer(index=(len(trained_model.layers)-1))
print(dense_6)

In [12]:
#This is the log sequence embedding from the last layer
# we can treat this as the features from the logs
dense_6.output

<tf.Tensor 'dense_1/Softmax:0' shape=(32, 4) dtype=float32>

In [13]:
# Then, we use the pre-trained model to extract intent features for 
# learning the decision boundary

In [14]:
class LogLineEncoder(tf.keras.Model):
    def __init__(self, num_of_conv1d=3,  
                 filters=64,
                 kernel_size=3, ):
        super().__init__()            
        self.num_of_conv1d = num_of_conv1d       
        self.filters = filters
        self.kernel_size = kernel_size           
        self.embedding_weights, self.vocab_size, self.char_onehot = get_embedding_layer(bglog)       
        
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size+1,
                                    output_dim=self.vocab_size,
                                    input_length=train_data.element_spec[0].shape[2],
                                    weights = [self.embedding_weights],
                                    )
        self.conv1d_layers = [tf.keras.layers.Conv1D(filters=filters, 
                                                kernel_size=kernel_size, 
                                                padding='same')  
                       for _ in range(self.num_of_conv1d)]
        self.maxpool2d = tf.keras.layers.MaxPooling2D(
            pool_size=(1, train_data.element_spec[0].shape[2]))
                  
        
    def call(self, inputs):
        x = self.embedding(inputs)
        for conv1d_layer in self.conv1d_layers:
            x = conv1d_layer(x)
        x = self.maxpool2d(x)
        x = tf.reshape(x, (inputs.shape[0], inputs.shape[1], self.filters))
        return x
    
    

# 
line_encoder =   LogLineEncoder()
# the model doesn't have a state unless it is called at least once
# in order to initialize the model we need a sample data 
sample_train_data = next(iter(train_data))
sample_x_train = sample_train_data[0]
print('sample_x_train.shape:', sample_x_train.shape)
# now we will initialize the model with the sample data
loglineEmbedding = line_encoder(sample_x_train)
print('loglineEmbedding.shape:', loglineEmbedding.shape)
# Now the model have a state and can be inspected        
line_encoder.summary()

vocab_size: 50
sample_x_train.shape: (32, 32, 64)
loglineEmbedding.shape: (32, 32, 64)
Model: "log_line_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  2550      
_________________________________________________________________
conv1d_6 (Conv1D)            multiple                  9664      
_________________________________________________________________
conv1d_7 (Conv1D)            multiple                  12352     
_________________________________________________________________
conv1d_8 (Conv1D)            multiple                  12352     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
Total params: 36,918
Trainable params: 36,918
Non-trainable params: 0
_________________________________________________________________


LOG SEQUENCE EMBEDDING TAKES LOGLINE EMBEDDING AS INPUT

In [55]:
class LogSeqEncoder(tf.keras.Model):
    
    def __init__(self, num_of_conv1d=3,  filters=64,
                 kernel_size=3, maxpool_1=True,
                 dense_neurons=16, dense_activation='relu',):
        super().__init__()
        self.num_of_conv1d = num_of_conv1d
        self.dense_neurons = dense_neurons
        self.filters = filters
        self.kernel_size = kernel_size
        self.maxpool_1 = maxpool_1
        self.dense_activation = dense_activation
        self.conv1d_layers = [tf.keras.layers.Conv1D(filters=filters, 
                                                kernel_size=kernel_size, 
                                                padding='same')  
                       for _ in range(self.num_of_conv1d)]
        self.maxpool1d = tf.keras.layers.MaxPooling1D(pool_size=(train_data.element_spec[0].shape[1]) )
        
        self.Dense = tf.keras.layers.Dense(self.dense_neurons, 
                                           activation=self.dense_activation)
       
        
    def call(self, inputs):       
        for conv1d_layer in self.conv1d_layers:
            x = conv1d_layer(inputs)
        x = self.maxpool1d(x)        
        x = tf.reshape(x, (inputs.shape[0], self.filters))
        x = self.Dense(x)
        return x
    
    

logSeqencer =   LogSeqEncoder()
# the model doesn't have a state unless it is called at least once
logSeqEmbedding = logSeqencer(loglineEmbedding)
print('logSeqEmbedding.shape:', logSeqEmbedding.shape)
# Now the model have a state and can be inspected        
logSeqencer.summary()

logSeqEmbedding.shape: (32, 16)
Model: "log_seq_encoder_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_33 (Conv1D)           multiple                  12352     
_________________________________________________________________
conv1d_34 (Conv1D)           multiple                  12352     
_________________________________________________________________
conv1d_35 (Conv1D)           multiple                  12352     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 multiple                  0         
_________________________________________________________________
dense_10 (Dense)             multiple                  1040      
Total params: 38,096
Trainable params: 38,096
Non-trainable params: 0
_________________________________________________________________


In [16]:
# sample_x_train

In [56]:
class LogClassifier(tf.keras.Model):
    
    def __init__(self,  **kwargs):
        super().__init__(**kwargs)
        self.log_line_encoder = LogLineEncoder()
        self.log_seq_encoder = LogSeqEncoder()
        self.classifier = tf.keras.layers.Dense(
            train_data.element_spec[1].shape[1], activation='softmax')
#         self.extract_feature = extract_feature
    
    def call(self, inputs, extract_feature=False,):
#         x_data, y_data = inputs
        x = self.log_line_encoder(inputs)
        seq_embedding = self.log_seq_encoder(x)
        
        if  extract_feature:
            output = seq_embedding
        else:
            output = self.classifier(seq_embedding)
        return output
    
log_classifier = LogClassifier()
log_classifier(sample_x_train)        

vocab_size: 50


<tf.Tensor: shape=(32, 4), dtype=float32, numpy=
array([[0.24365869, 0.22684833, 0.24284323, 0.2866498 ],
       [0.2387654 , 0.26823872, 0.21437998, 0.27861592],
       [0.24500129, 0.22858903, 0.22597234, 0.30043742],
       [0.24127483, 0.24924593, 0.23263992, 0.27683935],
       [0.24661951, 0.23611398, 0.24892668, 0.2683398 ],
       [0.25893798, 0.25272855, 0.23717998, 0.25115353],
       [0.25970316, 0.25312582, 0.23949133, 0.2476798 ],
       [0.2500926 , 0.25995392, 0.24081607, 0.2491374 ],
       [0.2573613 , 0.2393681 , 0.24555507, 0.25771552],
       [0.23853046, 0.23317362, 0.23815306, 0.29014286],
       [0.25994053, 0.25003916, 0.25392714, 0.23609312],
       [0.25100702, 0.21426399, 0.24512401, 0.289605  ],
       [0.272541  , 0.22799456, 0.24962693, 0.2498375 ],
       [0.25997856, 0.23823307, 0.24978384, 0.25200447],
       [0.23672302, 0.25411123, 0.21897729, 0.29018852],
       [0.24154194, 0.2587037 , 0.2184499 , 0.2813044 ],
       [0.25910193, 0.2438815 , 0.23756

In [18]:
# the classifier assigned low probability to all the classes since it is untrained
# TODO: the mode should accept a single sequence. At present it is accepting only a batch

In [57]:
log_classifier.summary()

Model: "log_classifier_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
log_line_encoder_3 (LogLineE multiple                  36918     
_________________________________________________________________
log_seq_encoder_5 (LogSeqEnc multiple                  38096     
_________________________________________________________________
dense_12 (Dense)             multiple                  68        
Total params: 75,082
Trainable params: 75,082
Non-trainable params: 0
_________________________________________________________________


In [58]:
# This is to check that the model's built in  complile and fit is working well
log_classifier.compile(optimizer='adam', 
                  loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
hist = log_classifier.fit(train_data, validation_data=test_data, epochs=1) 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
100/100 [==============================] - 9s 94ms/step - loss: 0.3766 - accuracy: 0.8866 - precision_4: 0.9643 - recall_4: 0.7672 - val_loss: 0.0154 - val_accuracy: 0.9985 - val_precision_4: 0.9985 - val_recall_4: 0.9940


In [59]:
# now after the training the predeicitoin will show higher probability to the 
# a class and lesser probability to other classes
log_classifier(sample_x_train)      

<tf.Tensor: shape=(32, 4), dtype=float32, numpy=
array([[7.66337962e-06, 9.80014011e-05, 1.00093067e-03, 9.98893440e-01],
       [9.89592433e-01, 5.51328994e-03, 3.35740834e-03, 1.53688435e-03],
       [6.62352366e-04, 3.03132948e-03, 5.61865091e-01, 4.34441179e-01],
       [4.07132698e-04, 2.33796425e-03, 9.93918657e-01, 3.33624077e-03],
       [7.37686287e-06, 1.34223315e-04, 1.77774008e-03, 9.98080611e-01],
       [9.91067469e-01, 8.64739716e-03, 7.38604649e-05, 2.11355509e-04],
       [9.91131008e-01, 8.56753252e-03, 7.96614768e-05, 2.21782393e-04],
       [9.91044044e-01, 8.64533335e-03, 7.55685614e-05, 2.35019528e-04],
       [3.33352759e-03, 9.96509612e-01, 1.23531336e-05, 1.44439924e-04],
       [7.12871997e-06, 7.86651581e-05, 1.06223300e-03, 9.98852015e-01],
       [1.31358288e-03, 9.98492002e-01, 1.73531407e-05, 1.76999994e-04],
       [2.50415440e-04, 5.75117883e-04, 9.96209264e-01, 2.96528218e-03],
       [5.69077267e-04, 1.86373445e-03, 1.38085589e-01, 8.59481573e-01],
  

In [60]:
features = log_classifier(sample_x_train, extract_feature=True ) 
features

<tf.Tensor: shape=(32, 16), dtype=float32, numpy=
array([[ 0.8232021 ,  5.750864  ,  5.5350776 ,  0.        ,  6.0789514 ,
         0.27661368,  2.9831824 ,  3.068651  ,  3.690295  ,  0.        ,
         0.06916624,  1.2565458 ,  6.966045  ,  0.        ,  4.735535  ,
         0.        ],
       [ 2.8035774 ,  0.33830366,  1.029506  ,  0.        ,  1.9357386 ,
         4.2382283 ,  4.4059677 ,  1.4008049 ,  3.6615927 ,  0.        ,
         0.        ,  4.04283   ,  0.85960007,  1.1615007 ,  6.68119   ,
         8.036921  ],
       [ 1.8879466 ,  4.6071744 ,  4.3146276 ,  0.        ,  6.5413814 ,
         0.        ,  1.8162975 ,  2.134408  ,  2.9991972 ,  0.        ,
         0.7637098 ,  1.9248446 ,  7.564635  ,  0.        ,  1.702782  ,
         1.2924783 ],
       [ 3.1914344 ,  3.6201518 ,  3.8094075 ,  0.        ,  7.528643  ,
         0.        ,  1.2003884 ,  1.3355631 ,  2.2037168 ,  0.        ,
         0.62469894,  3.254866  ,  7.6564665 ,  0.        ,  0.9608521 ,
        

In [61]:
np.zeros((2, 2))

array([[0., 0.],
       [0., 0.]])

In [62]:
for batch in train_data:
    x_train, y_train = batch
    print(x_train)
    print(y_train)
    break

tf.Tensor(
[[[14 13 13 ... 11 11 25]
  [13 13 22 ... 11 11 25]
  [14 13 13 ... 11 11 25]
  ...
  [10  8  4 ...  6  6  7]
  [13 13 13 ... 22 21 21]
  [13 13 13 ... 22 21 21]]

 [[ 2 19 11 ...  8 10  2]
  [ 2 19 11 ...  8 10  2]
  [ 2 19 11 ...  8 10  2]
  ...
  [ 2 19 11 ...  8 10  2]
  [ 2 19 11 ...  8 10  2]
  [ 2 19 11 ...  8 10  2]]

 [[18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  ...
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]]

 ...

 [[18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  ...
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]]

 [[18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  ...
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]]

 [[18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  ...
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]
  [18  2  4 ...  0  0  0]]]

In [68]:
centroids = np.zeros((train_data.element_spec[1].shape[1],   16))
print('centriods initialized:', centroids)
total_labels = np.zeros(train_data.element_spec[1].shape[1]) # it was 4
# total_labels[2] += 1
# total_labels[2] += 1
print('total_labels initialized:', total_labels)
for batch in train_data:
    logseq_batch, label_batch = batch
    features = log_classifier(logseq_batch, extract_feature=True )
    for i in range(len(label_batch)):
        label = label_batch[i]
        numeric_label = np.argmax(label)
        total_labels[numeric_label] += 1
        centroids[numeric_label] += features[i]
total_label_reshaped = np.reshape(total_labels, (train_data.element_spec[1].shape[1], 1))
centroids /= total_label_reshaped
print('centroids:',centroids)
print('total_labels:',total_label_reshaped)


centriods initialized: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
total_labels initialized: [0. 0. 0. 0.]
centroids: [[2.42723633e+00 1.61075535e-01 1.04813332e+00 0.00000000e+00
  9.88495178e-01 5.11170959e+00 3.71239746e+00 1.05664352e+00
  3.15759613e+00 0.00000000e+00 0.00000000e+00 4.53865234e+00
  4.20471764e-01 2.19021057e+00 8.06535583e+00 8.36452026e+00]
 [2.22617950e+00 1.72305374e+00 4.27698151e+00 0.00000000e+00
  4.17190155e+00 2.86426941e+00 1.52251129e-01 3.54656738e+00
  9.93699341e-01 0.00000000e+00 0.00000000e+00 5.45192078e+00
  5.91299553e-01 1.07752678e+00 1.09522058e+01 2.66130615e+00]
 [2.81991516e+00 3.66816589e+00 3.50354187e+00 0.00000000e+00
  7.68100098e+00 5.91734838e-02 2.02764847e+00 1.43781494e+00
  2.28084656e+00 0.00000000e+00 9.29355774e-01 2.48236298e+00
  7.43869141e+00 1.99984103e-03 1.276

In [65]:
# to understand the np divide operation
n1 = np.arange(12)
n2 = n1.reshape((3, 4))
print('n2', n2)
n3 = np.array([2, 2, 2])
print('n3', n3)
n4 = np.reshape(n3, (3, 1))
print('n4', n4)
n2/n4


n2 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
n3 [2 2 2]
n4 [[2]
 [2]
 [2]]


array([[0. , 0.5, 1. , 1.5],
       [2. , 2.5, 3. , 3.5],
       [4. , 4.5, 5. , 5.5]])

In [66]:
#take zi and a ck 
# from sample_x_train the first sample belongs to class 3
print('sample_x_train[0]', sample_x_train[0])
sample_y_train = sample_train_data[1]
print('sample_y_train[0]', sample_y_train[0])
print('feature for the same:', features[0])
print('centroid for the class 3 :', centroids[3])

sample_x_train[0] tf.Tensor(
[[ 4 30 25 ...  9  0  0]
 [30 25 17 ...  8  4 14]
 [12  3  6 ... 12  6 18]
 ...
 [30 25 17 ...  8  4 14]
 [12  3  6 ... 12  6 18]
 [ 4 30 25 ...  9  0  0]], shape=(32, 64), dtype=int32)
sample_y_train[0] tf.Tensor([0. 0. 0. 1.], shape=(4,), dtype=float32)
feature for the same: tf.Tensor(
[2.8337116 4.522587  4.577522  0.        7.5018716 0.        2.0851574
 1.8526348 2.728645  0.        0.7425541 2.766746  7.8422027 0.
 2.1378212 1.855924 ], shape=(16,), dtype=float32)
centroid for the class 3 : [0.98276848 5.30186157 5.08779053 0.         6.05381165 0.11221761
 2.69602173 2.98188934 3.56977234 0.         0.11795975 1.40428314
 6.79935242 0.01297427 4.24655457 0.37033184]


In [67]:
# eucladian distance
z_0_3= features[0] # [16], earlier [2048]
C_3 = centroids[3] # [16], earlier [2048]
ED = np.sum(np.square(z_0_3 - C_3 ))
print('eucladian distance:', ED)
# InvalidArgumentError: Incompatible shapes: [32,64] vs. [2048] [Op:Sub]


eucladian distance: 18.746838


In [88]:
# wwe can not substract different shaped arrays , see the error
print('features.shape', features.shape)
print('centroids.shape', centroids.shape)
# features - centroids # InvalidArgumentError: Incompatible shapes: [32,16] vs. [4,16] [Op:Sub]
z = np.expand_dims(features, axis=1)
C =  np.expand_dims(centroids, axis=0)
print('np.expand_dims(features, axis=1) :', z.shape)
print('np.expand_dims(centroids, axis=0):', C.shape)
# Now we can substract
sub_z_C = z - C
print('sub_z_C', sub_z_C.shape)
squred_sum = np.sum(np.square(z- C), axis=2)
print('squred_sum', squred_sum.shape)

features.shape (32, 16)
centroids.shape (4, 16)
np.expand_dims(features, axis=1) : (32, 1, 16)
np.expand_dims(centroids, axis=0): (1, 4, 16)
sub_z_C (32, 4, 16)
squred_sum (32, 4)


In [98]:
# make the dimensions same for substraction 
def euclidean_metric(a, b):
#     n = a.shape[0]
#     m = b.shape[0]
    a = np.expand_dims(a, 1)
    b = np.expand_dims(b, 0)
#     logits = -((a - b)**2).sum(dim=2)
    logits = -np.sum(np.square(a - b), axis=2)
    return logits  
ED_logits = euclidean_metric(features, centroids)
print('ED_logits', ED_logits.shape)
print('ED_logits_sample', ED_logits[0])

ED_logits (32, 4)
ED_logits_sample [-201.07862778  -24.55007728 -211.70321366 -148.03531404]


In [112]:
# smax = tf.nn.softmax(ED_logits, axis=1)
smax = tf.nn.softmax(ED_logits, )
print('smax.shape:', smax.shape)
print('smax_sample:', smax[0])
reduced_max = tf.reduce_max(smax, axis=1)
print('reduced_max:', reduced_max)

smax.shape: (32, 4)
smax_sample: tf.Tensor([2.16085000e-77 1.00000000e+00 5.25322016e-82 2.34986590e-54], shape=(4,), dtype=float64)
reduced_max: tf.Tensor(
[1.         1.         1.         1.         1.         1.
 1.         0.99998873 1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.99986173 1.         1.         1.         1.
 1.         1.        ], shape=(32,), dtype=float64)


In [ ]:
class BoundaryLoss(tf.keras.layers.Layer):
    def __init__(self, num_labels=train_data.element_spec[1].shape[1], 
                feat_dim = 16):
        super().__init__()
        self.num_labels = num_labels
        self.feat_dim = feat_dim
        w_init = tf.random_normal_initializer()
        self.delta = tf.Variable(
            initial_value=w_init((self.num_labels), dtype="float32"),
            trainable=True,
        )
        
    def call(self, features, centroids, labels):
        logits =  euclidean_metric(features, centroids)  
        ######### Why softmax before softplus#########
        smax = tf.nn.softmax(logits, )
        reduced_max = tf.reduce_max(smax, axis=1)
        ############################
        delta = tf.nn.softplus(self.delta)
        label_indexs = np.argmax(labels)
        c = centroids[label_indexs]
        d = delta[label_indexs]
        x = features
        
        euc_dis = tf.norm(x - c, ord='euclidean', axis=1,)
        ##If axis is None (the default), the input is considered a vector and a 
        ## single vector norm is computed over the entire set of values in the tensor, 
        ## i.e. norm(tensor, ord=ord) is equivalent to norm(reshape(tensor, [-1]), ord=ord). If axis is a Python integer, the input is considered a batch of vectors, and axis determines the axis in tensor over which to compute vector norms.
        
        
        

In [ ]:
# understand this , a= features(batch_size, 2048) , b = centroids (4, 2048)
def euclidean_metric_torch(a, b):
    n = a.shape[0]
    m = b.shape[0]
    a = a.unsqueeze(1).expand(n, m, -1)
    b = b.unsqueeze(0).expand(n, m, -1)
    logits = -((a - b)**2).sum(dim=2)
    return logits

In [81]:
import numpy as np
import tensorflow as tf
a = np.arange(6)
a = a.reshape((2, -1))
print('a:', a)
print('a.shape', a.shape)
b = np.arange(8, 16)
print('b',b)
b = np.reshape(b, (4, -1))
print('b',b)
print('b.shape:',b.shape)
tfa = tf.constant(a)
tfb = tf.constant(b)
print('tfa',tfa)
print('tfb',tfb)
# n = tfa.shape[0]
# m = b.shape[0]


a: [[0 1 2]
 [3 4 5]]
a.shape (2, 3)
b [ 8  9 10 11 12 13 14 15]
b [[ 8  9]
 [10 11]
 [12 13]
 [14 15]]
b.shape: (4, 2)
tfa tf.Tensor(
[[0 1 2]
 [3 4 5]], shape=(2, 3), dtype=int32)
tfb tf.Tensor(
[[ 8  9]
 [10 11]
 [12 13]
 [14 15]], shape=(4, 2), dtype=int32)


In [78]:
print('tf.expand_dims(tfa, 0) :',tf.expand_dims(tfa, 0))
print()
print('tf.expand_dims(tfa, 1) :',tf.expand_dims(tfa, 1))
print()
print('tf.expand_dims(tfa, 1) :',tf.expand_dims(tfa, -1))

tf.expand_dims(tfa, 0) : tf.Tensor(
[[[0 1 2]
  [3 4 5]]], shape=(1, 2, 3), dtype=int32)

tf.expand_dims(tfa, 1) : tf.Tensor(
[[[0 1 2]]

 [[3 4 5]]], shape=(2, 1, 3), dtype=int32)

tf.expand_dims(tfa, 1) : tf.Tensor(
[[[0]
  [1]
  [2]]

 [[3]
  [4]
  [5]]], shape=(2, 3, 1), dtype=int32)


In [82]:
tfa = tf.expand_dims(tfa, 1)
print(f'tf.shape(tfa): {tf.shape(tfa)}')
tfb = tf.expand_dims(tfb, 0)
print(f'tf.shape(tfb): {tf.shape(tfb)}')

tf.shape(tfa): [2 1 3]
tf.shape(tfb): [1 4 2]


In [86]:
n = a.shape[0]
m = b.shape[0]
tfa_broadcast = tf.broadcast_to(tfa, [2, 4, 3])
tf.shape(tfa_broadcast)
print('tfa_broadcast',tfa_broadcast)

tfa_broadcast tf.Tensor(
[[[0 1 2]
  [0 1 2]
  [0 1 2]
  [0 1 2]]

 [[3 4 5]
  [3 4 5]
  [3 4 5]
  [3 4 5]]], shape=(2, 4, 3), dtype=int32)


In [ ]:
# 6

# The equivalent function for pytorch expand is tensorflow tf.broadcast_to

# Docs: https://www.tensorflow.org/api_docs/python/tf/broadcast_to

# Share
# Follow
# edited Oct 23, 2021 at 18:22

# M.Innat
# 12.2k66 gold badges3434 silver badges6767 bronze badges
# answered Jan 4, 2019 at 9:12

# funkyyyyyy
# 6111 silver badge22 bronze badges
# Add a comment

# 0

# Tensorflow automatically broadcasts, so in general you don't need to do any of this. Suppose you have a y' of shape 6x2x3 and your x is of shape 2x3, then you can already do y'*x or y'+x will already behave as if you had expanded it. But if for some other reason you really need to do it, then the command in tensorflow is tile:

# y = tf.tile(tf.reshape(x, (1,2,3)), multiples=(6,1,1))
# Docs: https://www.tensorflow.org/api_docs/python/tf/tile

In [ ]:
def euclidean_metric(a, b):
    n = a.shape[0]
    m = b.shape[0]
    a = tf.expand_dims(a, 1)
    b = tf.expand_dims(b, 0)
    logits = -((a - b)**2).sum(dim=2)
    return logits  

In [87]:
class OpenSet:
    def __init__(self, data, pretrained_model):
        
        self.model = pretrained_model
        self.best_eval_score = 0
        self.delta = None
        self.delta_points = []
        self.centroids = None
        self.test_results = None
        self.predictions = None
        self.true_labels = None
        
    def centroids_cal(self):
        centriods = np.zeros(train_data.element_spec[1].shape[1], embedding_size)
        total_labels = np.empty(0, dtype=longdouble)
        


In [ ]:
# Customizing the training step to get centroid for each class
class OpenSet:
    def __init__(self, data, pretrained_model=log_classifier):
#         super().__init__():
        self.model = pretrained_model        
        self.centroids = None
        self.num_labels = train_data.element_spec[1].shape[1]
        
    def centroids_cal(self):
        centriods = np.zeros(self.num_labels, embedding_size)
        total_labels = np.empty(0, dtype=longdouble)
        for batch in data:
            logseq_batch, label_batch = batch
            features = self.model(logseq_batch, extract_feature=True ) 
            

In [ ]:
# In context of deep learning the logits layer means the layer that feeds in to softmax (or other such normalization). The output of the softmax are the probabilities for the classification task and its input is logits layer. The logits layer typically produces values from -infinity to +infinity and the softmax layer transforms it to values from 0 to 1.

# Historical Context

# Where does this term comes from? In 1930s and 40s, several people were trying to adapt linear regression to the problem of predicting probabilities. However linear regression produces output from -infinity to +infinity while for probabilities our desired output is 0 to 1. One way to do this is by somehow mapping the probabilities 0 to 1 to -infinity to +infinity and then use linear regression as usual. One such mapping is cumulative normal distribution that was used by Chester Ittner Bliss in 1934 and he called this "probit" model, short for "probability unit". However this function is computationally expensive while lacking some of the desirable properties for multi-class classification. In 1944 Joseph Berkson used the function log(p/(1-p)) to do this mapping and called it logit, short for "logistic unit". The term logistic regression derived from this as well.

# The Confusion

# Unfortunately the term logits is abused in deep learning. From pure mathematical perspective logit is a function that performs above mapping. In deep learning people started calling the layer "logits layer" that feeds in to logit function. Then people started calling the output values of this layer "logit" creating the confusion with logit the function.